# building cross-compiler GNU toolchain

Cross-compiler build is going using GNU [make](make.ipynb) tool, which is the minimal and still powerful automation tool for build control.

## target system configuration

configuration variables can be overriden in a command line:
```
cd msp430 ; make APP=cnc HW=pc104 TARGET=i686 binutils
```

### application

In [504]:
%%writefile msp430/Makefile

APP    ?= blink

Overwriting msp430/Makefile


### hardware device

Hardware device you are targetting, must be used for lot of configuration: GPIO ports, peripherial pingout,..

* `HW`
  * [msp430g2]()
  * [cm0pill]()
  * [cm1pill]()

In [505]:
%%writefile -a msp430/Makefile

HW     ?= msp430g2

Appending to msp430/Makefile


### target MCU/CPU/OS triplet

* `BUILD`
  - system name of your workstation OS in form of **system triplet**
    - `x86_64-pc-linux-gnu`
* `HOST`
  - triplet of OS will run on an emLinux embedded device ([Raspberry](), [PC104](),..)
    * for MCU targets we'll drop it
* `TARGET`
  - triplet of MCU target
    * [msp430](msp430.ipynb)
    * i386

In [506]:
%%writefile -a msp430/Makefile

TARGET ?= msp430

Appending to msp430/Makefile


## directory structure

This is set of directories will be created 

* `CWD` in the same dir as `Makefile` itself or directory selected by `make -C` option
* `GZ` downloaded archives
* `TMP` temp build directory
* `SRC` source code for every tool will be built
* `GNU` all compilers and tools will be pushed locally into this directory
  - such a scheme will not affect to whole `BUILD`/`HOST` system, all toolds will reside in a user `HOME`

In [507]:
%%writefile -a msp430/Makefile

CWD = $(CURDIR)
GZ  = $(CWD)/gz
TMP = $(CWD)/tmp
SRC = $(TMP)/src
GNU = $(CWD)/gnu_$(TARGET)

Appending to msp430/Makefile


In [508]:
%%writefile -a msp430/Makefile

all: dirs gz binutils gdb

Appending to msp430/Makefile


In [509]:
%%writefile -a msp430/Makefile

dirs:
	mkdir -p $(GZ) $(TMP) $(SRC) $(GNU)

Appending to msp430/Makefile


In [510]:
%%sh
cd msp430 ; make dirs ; ls -la ; ls -la tmp

mkdir -p /home/dpon/jupy/msp430/gz /home/dpon/jupy/msp430/tmp /home/dpon/jupy/msp430/tmp/src /home/dpon/jupy/msp430/gnu_msp430
итого 36
drwxr-xr-x  6 dpon dpon 4096 май 29 17:38 .
drwxr-xr-x 11 dpon dpon 4096 май 29 17:58 ..
-rw-r--r--  1 dpon dpon  724 май 29 15:04 cross.ipynb
-rw-r--r--  1 dpon dpon   21 май 29 16:41 .gitignore
drwxr-xr-x  5 dpon dpon 4096 май 29 17:36 gnu_msp430
drwxr-xr-x  2 dpon dpon 4096 май 29 17:51 gz
drwxr-xr-x  2 dpon dpon 4096 май 29 14:41 .ipynb_checkpoints
-rw-r--r--  1 dpon dpon  220 май 29 18:00 Makefile
drwxr-xr-x  5 dpon dpon 4096 май 29 17:58 tmp
итого 20
drwxr-xr-x  5 dpon dpon 4096 май 29 17:58 .
drwxr-xr-x  6 dpon dpon 4096 май 29 17:38 ..
drwxr-xr-x 12 dpon dpon 4096 май 29 17:35 binutils-2.32
drwxr-xr-x  2 dpon dpon 4096 май 29 17:58 gdb-8.3
drwxr-xr-x  4 dpon dpon 4096 май 29 17:52 src


## cross-compiler components

## versions

In [511]:
%%writefile -a msp430/Makefile

BINUTILS_VER = 2.32
GDB_VER      = 8.3

Appending to msp430/Makefile


In [512]:
%%writefile -a msp430/Makefile

BINUTILS = binutils-$(BINUTILS_VER)
GDB      = gdb-$(GDB_VER)

Appending to msp430/Makefile


## predownload all archives

In [513]:
%%writefile -a msp430/Makefile

BINUTILS_GZ = $(BINUTILS).tar.xz
GDB_GZ      = $(GDB).tar.xz

Appending to msp430/Makefile


In [514]:
%%writefile -a msp430/Makefile

WGET = wget -c -P $(GZ) -q

gz: $(GZ)/$(BINUTILS_GZ)

Appending to msp430/Makefile


In [515]:
%%sh
cd msp430 ; make gz ; ls -la gz

make: Цель «gz» не требует выполнения команд.
итого 40308
drwxr-xr-x 2 dpon dpon     4096 май 29 17:51 .
drwxr-xr-x 6 dpon dpon     4096 май 29 17:38 ..
-rw-r--r-- 1 dpon dpon 20774880 фев  2 20:38 binutils-2.32.tar.xz
-rw-r--r-- 1 dpon dpon 20490300 май 11 22:47 gdb-8.3.tar.xz


### archive unpack rules

In [516]:
%%writefile -a msp430/Makefile

$(SRC)/%/README: $(GZ)/%.tar.xz
	cd $(SRC) ; xzcat $< | tar -x && touch $@

Appending to msp430/Makefile


### configure scripts run

In [517]:
%%writefile -a msp430/Makefile

XPATH = PATH=$(GNU)/bin:$(PATH)

CFG = configure --disable-nls --prefix=$(GNU)

MAKE = $(XPATH) make

Appending to msp430/Makefile


most components can be built with use of full power of your computer, but `gcc` build can't be done in parallel even on i7/6G so `make gcc0` **will be run in single process** -- run in in background, and do some useful work while building: read docs, take a rest with tee, etc

In [518]:
%%writefile -a msp430/Makefile

CPU_CORES = $(shell grep processor /proc/cpuinfo|wc -l)

MAKEJ = $(MAKE) -j$(CPU_CORES)

Appending to msp430/Makefile


### [binutils](binutils.ipynb) : assembler, linker, object file tools

In [519]:
%%writefile -a msp430/Makefile

binutils: $(GNU)/bin/$(TARGET)-ld

CFG_BINUTILS += --target=$(TARGET)

$(GNU)/bin/$(TARGET)-ld: $(GZ)/$(BINUTILS_GZ) $(SRC)/$(BINUTILS)/README
	rm -rf $(TMP)/$(BINUTILS) ; mkdir $(TMP)/$(BINUTILS) ; cd $(TMP)/$(BINUTILS) ;\
	$(XPATH) $(SRC)/$(BINUTILS)/$(CFG) $(CFG_BINUTILS) && $(MAKEJ) && $(MAKE) install-strip

$(GZ)/$(BINUTILS_GZ):
	$(WGET) https://mirror.tochlab.net/pub/gnu/binutils/$(BINUTILS_GZ)

Appending to msp430/Makefile


In [520]:
%%sh
# cd msp430 ; make binutils ; ls -la tmp/src tmp gnu*

after `make` run `configure` will be run with required options:

`/home/dpon/jupy/msp430/tmp/src/binutils-2.32/configure --disable-nls --prefix=/home/user/jupy/msp430 --target=msp430`

detecting host/target

```
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking target system type... msp430-unknown-none
```

build requires some preinstalled developmen packages:
 * `sudo apt install`
   - `build-essential` `m4` `wget` `make` `gcc g++` `flex` `bison`
   - `xz` `bzip2`
   - `bc` `libncurses-dev` only for Linux kernel build

```
checking for gcc... gcc
checking for g++... g++
checking for bison... bison
checking for m4... m4
checking for flex... flex
```

`make -j4`

`make install-strip`

### [gdb](gdb.ipynb) : remote hardware debugger

In [521]:
%%writefile -a msp430/Makefile

gdb: $(GNU)/bin/$(TARGET)-gdb

CFG_GDB += $(CFG_BINUTILS) --enable-python

$(GNU)/bin/$(TARGET)-gdb: $(GZ)/$(GDB_GZ) $(SRC)/$(GDB)/README
	rm -rf $(TMP)/$(GDB) ; mkdir $(TMP)/$(GDB) ; cd $(TMP)/$(GDB) ;\
	$(XPATH) $(SRC)/$(GDB)/$(CFG) $(CFG_GDB) && $(MAKEJ) && $(MAKE) install-strip

$(GZ)/$(GDB_GZ):
	$(WGET) http://ftp.gnu.org/gnu/gdb/$(GDB_GZ)

Appending to msp430/Makefile
